In [15]:
import pandas as pd
import numpy as np

train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')
submission=pd.read_csv('sample_submission.csv')

In [9]:
print(train.columns)

Index(['date_time', 'wind_direction', 'sky_condition', 'precipitation_form',
       'wind_speed', 'humidity', 'low_temp', 'high_temp',
       'Precipitation_Probability', 'number_of_rentals'],
      dtype='object')


In [88]:
import pandas as pd
import numpy as np

train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')
submission=pd.read_csv('sample_submission.csv')


import xgboost as xgb

# 데이터 로드 및 분할

def year(date_time):
    return int(date_time[:4])

def month(date_time):
    return int(date_time[5:7])

#적용
train['year'] = train['date_time'].apply(year)
test['year'] = test['date_time'].apply(year)

train['month'] = train['date_time'].apply(month)
test['month'] = test['date_time'].apply(month)

#date_time 제거
train=train.drop(['date_time'], axis=1)
test=test.drop(['date_time'], axis=1)

X=train.drop(columns=['number_of_rentals', 'wind_direction'])
test=test.drop(columns=['wind_direction'])
y=train['number_of_rentals']

from sklearn.model_selection import KFold
kf = KFold(n_splits=5, shuffle=True, random_state=42)
for train_idx, valid_idx in kf.split(X):
    X_train, X_valid = X.iloc[train_idx], X.iloc[valid_idx]
    y_train, y_valid = y.iloc[train_idx], y.iloc[valid_idx]

# 모델 설정 및 학습
model = xgb.XGBRegressor(objective='reg:squarederror', max_depth=3, learning_rate=0.1, n_estimators=50)
model.fit(X_train, y_train)

# 예측 수행
train_pred=model.predict(X_train)
valid_pred=model.predict(X_valid)
test_pred=model.predict(test)

#평가
def nmae(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true))

train_nmae = nmae(y_train, train_pred)
valid_nmae = nmae(y_valid, valid_pred)

print(f"Train NMAE: {train_nmae:}")
print(f"Validation NMAE: {valid_nmae:}")


from sklearn.metrics import r2_score
train_r2 = r2_score(y_train, train_pred)
print(f"train r2 score: {train_r2}")
valid_r2= r2_score(y_valid, valid_pred)
print(f"valid r2 score: {valid_r2}")

submission['number_of_rentals']=test_pred
submission.to_csv('XGB2_submission.csv', index=False)

Train NMAE: 0.09773892587545559
Validation NMAE: 0.21523573791214984
train r2 score: 0.9703631401062012
valid r2 score: 0.8248942494392395


In [35]:
import os
import pandas as pd

from sklearn.model_selection import GridSearchCV

# XGBRegressor 모델 설정
model = xgb.XGBRegressor()

# 그리드 서치 하이퍼파라미터 설정
param_grid = {
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.05, 0.1],
    'n_estimators': [50, 100, 200],
}

####
from sklearn.metrics import make_scorer

def nmae_score(y_true, y_pred):
    y_true = np.where(y_true == 0, 1, y_true)  # 0 나눗셈 방지
    return -np.mean(np.abs((y_true - y_pred) / y_true))  # GridSearch용 음수 반환

nmae_scorer = make_scorer(nmae_score, greater_is_better=False)

grid_search = GridSearchCV(estimator=model, param_grid=param_grid,
                           scoring=nmae_scorer, cv=5, verbose=1, n_jobs=1)

# 모델 학습
grid_search.fit(X_train, y_train)

# 최적의 하이퍼파라미터 출력
print("Best Parameters:", grid_search.best_params_)
print("Best Score:", grid_search.best_score_)



Fitting 5 folds for each of 27 candidates, totalling 135 fits
Best Parameters: {'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 50}
Best Score: 0.6063292935071838
